We start by importing all the libraries we are interested in.

In [1]:
import numpy as np
import astropy
import ccdproc
from ccdproc import CCDData, combiner
from astropy import units as u
import matplotlib.pyplot as plt 
from matplotlib.colors import LogNorm
import gc                                 # What does this library do?
gc.enable()

Load in the images.

In [7]:
images = ccdproc.ImageFileCollection("./C11_2022_03_28")    # loads in all of the image files
print(len(images.files))                                    # prints the number of files so we can make sure they have been loaded

326


# BIAS

We start by filtering the images to get the bias images

In [ ]:
# the same as above, but this time stores the filenames list to a variable and prints the list using the variable name
print('Printing a list of file names')
filenames = ( images.files_filtered(PICTTYPE = 2) )
print(filenames)

In [ ]:
biases = [ CCDData.read(fn, unit = "adu") for fn in images.files_filtered(PICTTYPE = 2) ]

print(len(biases), ' bias images loaded')

In [ ]:
# uses numpy to compute the min, max, standard deviation, and mean of the first bias image
print('Min:', np.min(biases[0]))
print('Max:', np.max(biases[0]))
print('Standard deviation:', np.std(biases[0]))
print('Mean:', np.mean(biases[0]))

In [ ]:
fig, ax = plt.subplots(figsize = (10,10))
plt.imshow(biases[0], cmap = 'Greys', norm = LogNorm())
plt.title('Bias Image 0')
plt.xlabel('x')
plt.ylabel('y')
plt.colorbar()
plt.show()

In [ ]:
bias_median = ccdproc.Combiner(biases, dtype=np.float32).median_combine()

# A good habit to get into is printing the statistics of input and output images
print('Image statistics for the median bias')

thisimage=bias_median
print('Min:', np.min(thisimage))
print('Max:', np.max(thisimage))
print('Median:', np.median(thisimage))
print('Std Dev:', np.std(thisimage))


In [ ]:
fig, ax = plt.subplots(figsize = (10,10))
plt.imshow(bias_median, cmap = 'Greys', norm = LogNorm())
plt.title('Median combined bias images')
plt.xlabel('x')
plt.ylabel('y')
plt.colorbar()
plt.show()

In [ ]:
bias_median.meta.update(EXPTIME = 0)                              # add the exposure time
bias_median.meta.update(TELESCOP = biases[0].header['TELESCOP'])  # add the telescope used
bias_median.meta.update(OBJECT = 'Bias_Median')                   # add the object name to identify this image as the median combined bias image
print(bias_median.meta)

In [ ]:
bias_median.write("bias_median.fits")     # writes the median combined bias image to a FITS file called 'bias_median.fits'

In [ ]:
# delete the bias data from the jupyter notebook to clear the memory
del(biases)
collected = gc.collect()
print('Check garbage collection', collected)

# DARK

In [ ]:
# find the filenames in the current directory that contain 'Dark_' at the start of the filename
images = ccdproc.ImageFileCollection(".",glob_include = 'Dark_*')

# iterate through the filenames, checking that PICTTYPE = 3 (which presumably indicates dark images)
for fn in images.files_filtered(PICTTYPE = 3):
    print(fn)
    
# loops through the filenames and reads the data for each file
darks = [ CCDData.read(fn, unit = "adu") for fn in images.files_filtered(PICTTYPE = 3) ]

In [ ]:
image = darks[0]
print('Image statistics for the 0th dark image')

print('Counts (ADU) for pixels 500 to 509 in column 500')
print(image[500:510,500])

print('Exposure time:',image.header['EXPTIME'], 'seconds')

print('Min:', np.min(image))
print('Max:', np.max(image))
print('Median:', np.median(image))
print('Std Dev:', np.std(image))

In [ ]:
for idx, thisimage in enumerate(darks): 
    darks[idx] = ccdproc.subtract_bias(thisimage, bias_median)

In [ ]:
image = darks[0]
print('Image statistics for the 0th dark image')

print('Counts (ADU) for pixels 500 to 509 in column 500')
print(image[500:510,500])

print('Exposure time:',image.header['EXPTIME'], 'seconds')

print('Electrons per minute for pixels 500 to 509 in column 500')
print(np.array(image[500:510,500])/image.header['EXPTIME']*image.header['EGAIN']*60)

print('Min:', np.min(image))
print('Max:', np.max(image))
print('Median:', np.median(image))
print('Std Dev:', np.std(image))

fig, ax = plt.subplots(figsize = (10,10))
plt.imshow(image, cmap = 'Greys', norm = LogNorm())
plt.title('Dark image 0 with bias subtracted')
plt.xlabel('x')
plt.ylabel('y')
plt.colorbar()
plt.show()

In [ ]:
dark_median = ccdproc.Combiner(darks, dtype=np.float32).median_combine() # median combine the dark images

In [ ]:
print('Statistics for median-combined dark image')

print('Counts (ADU) for pixels 500 to 509 in column 500')
print(dark_median[500:510,500])

print('Electrons per minute for pixels 500 to 509 in column 500')
print(np.array(dark_median[500:510,500])/image.header['EXPTIME']*image.header['EGAIN']*60)

print('Min:', np.min(dark_median))
print('Max:', np.max(dark_median))
print('Median:', np.median(dark_median))
print('Std Dev:', np.std(dark_median))

fig, ax = plt.subplots(figsize = (10,10))
plt.imshow(dark_median, cmap = 'Greys', norm = LogNorm())
plt.title('Median dark with bias subtracted')
plt.xlabel('x')
plt.ylabel('y')
plt.colorbar()
plt.show()

In [ ]:
dark_median.meta.update(EXPTIME = 3600)           # add exposure time to dark_median header
dark_median.meta.update(TELESCOP = 'C14')         # add telescope used
dark_median.meta.update(OBJECT = 'Dark_Median')   # add object name so we can identify the dark median image
dark_median.write("dark_median.fits")             # write the dark_median image to a fits file

In [ ]:
del(darks)
collected = gc.collect()
print('Check garbage collection', collected)

# FLATS

In [ ]:
images = ccdproc.ImageFileCollection(".")
for fn in images.files_filtered(PICTTYPE = 4):
    print(fn)

In [ ]:
# load in the R-band flats
images = ccdproc.ImageFileCollection(".",glob_include = 'Flat_R_*')
for fn in images.files_filtered(PICTTYPE = 4):
    print(fn)
flats = [ CCDData.read(fn, unit = "adu") for fn in images.files_filtered(PICTTYPE = 4) ]


In [ ]:
print('First column of flatss')
print(flats[0][:,0])

fig, ax = plt.subplots(figsize = (10,10))
plt.imshow(flats[0], cmap = 'Greys', norm = LogNorm())
plt.title('Flat 0')
plt.xlabel('x')
plt.ylabel('y')
plt.colorbar()
plt.show()

In [ ]:
print('First column of raw first flat image:')
print(flats[0][:,0])

# loop through flats and subtract bias from each of them, and overwrite the counts with the bias-subtracted values
for idx, thisimage in enumerate(flats): 
    flats[idx] = ccdproc.subtract_bias(thisimage, bias_median)
print('First column of first flat image after bias subtraction:')
print(flats[0][:,0])

# loop through flats and subtract darks, scaled by exposure time, for each of them
# then overwrite the counts with the dark-subtracted values
for idx, thisimage in enumerate(flats): 
    flats[idx] = ccdproc.subtract_dark(thisimage, dark_median, exposure_time = 'EXPTIME',
                                       exposure_unit = u.second, scale = True)
print('First column of first flat image after dark subtraction:')
print(flats[0][:,0]) 

In [ ]:
for thisimage in flats:
    print('Median:', np.ma.median(thisimage.data))  

In [ ]:
tempimages = flats.copy() # create a copy of the flat images

for idx, thisimage in enumerate(tempimages):    # loop through the copies of the flat images
    m = 1.0 / np.ma.median(tempimages[idx])     # calculate a normalisation constant by dividing 1 by the median value of the current flat image
    tempimages[idx] = tempimages[idx].multiply(m * u.adu)    # multiply the counts by the normalisation constant so that the median should now be 1

# median combine the normalised flat images    
FlatR_median = ccdproc.Combiner(tempimages, dtype=np.float32).median_combine()

In [ ]:
print('Statistics for median-combined flat image')

print(FlatR_median)

print('Min:', np.min(FlatR_median))
print('Max:', np.max(FlatR_median))
print('Median:', np.median(FlatR_median))
print('Std Dev:', np.std(FlatR_median))

fig, ax = plt.subplots(figsize = (10,10))
plt.imshow(FlatR_median, cmap = 'Greys', norm = LogNorm())
plt.title('Median-combined flat')
plt.xlabel('x')
plt.ylabel('y')
plt.colorbar()
plt.show()

In [ ]:
FlatR_median.meta.update(EXPTIME = 1)        # add exposure time in seconds to header
FlatR_median.meta.update(TELESCOP = 'C14')   # add telescope used
FlatR_median.meta.update(OBJECT = 'Flat_R_Median')   # add name of object/image
FlatR_median.write("Flat_R_median.fits")    # save to file

In [ ]:
# clear up some memory by deleting the flats and temporary images
del(flats)
del(tempimages)
collected = gc.collect()
print('Check garbage collection', collected)